In [1]:
# Homework 3 for Udacity CS344 Course, Intro to Parallel Programming
# clone the code repo,
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

#!git clone https://github.com/depctg/udacity-cs344-colab
!pip install git+https://github.com/depctg/nvcc4jupyter.git

# load cuda plugin
%config NVCCPluginV2.static_dir = True
%config NVCCPluginV2.relative_dir = "/content/drive/MyDrive/udacity-cs344-colab/src/HW3"

%load_ext nvcc_plugin

# change to work directory, generate makefiles
!mkdir ./udacity-cs344-colab/build
%cd ./udacity-cs344-colab/build
!cmake ../src
!rm -rf /content/udacity-cs344-colab

Mounted at /content/drive
/content/drive/MyDrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/depctg/nvcc4jupyter.git to /tmp/pip-req-build-0jbla6r1
  Running command git clone --filter=blob:none --quiet https://github.com/depctg/nvcc4jupyter.git /tmp/pip-req-build-0jbla6r1
  Resolved https://github.com/depctg/nvcc4jupyter.git to commit 63b811d987446c2a3783845b7b85cda6e562b71a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4331 sha256=3d6a5db7a997b5fb7129153949bb079aa3c5f8be792f58bc398ddf69019d2c1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-d34ylv42/wheels/8f/75/54/2711573f313166242b714906324ce0ed8321587fa2b11af05b
Successfully built NVCCPlugin
Set up cuda environment at /content/drive/MyDrive/udacity-cs344-colab/src/HW3
Default out bin result.out
mkdir: cannot create directory ‘./udacity-cs344-colab/build’: File exists
/con

In [7]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


Udacity Homework 3
   HDR Tone-mapping

  Background HDR
  ==============

  A High Dynamic Range (HDR) image contains a wider variation of intensity
  and color than is allowed by the RGB format with 1 byte per channel that we
  have used in the previous assignment.

  To store this extra information we use single precision floating point for
  each channel.  This allows for an extremely wide range of intensity values.

  In the image for this assignment, the inside of church with light coming in
  through stained glass windows, the raw input floating point values for the
  channels range from 0 to 275.  But the mean is .41 and 98% of the values are
  less than 3!  This means that certain areas (the windows) are extremely bright
  compared to everywhere else.  If we linearly map this [0-275] range into the
  [0-255] range that we have been using then most values will be mapped to zero!
  The only thing we will be able to see are the very brightest areas - the
  windows - everything else will appear pitch black.

  The problem is that although we have cameras capable of recording the wide
  range of intensity that exists in the real world our monitors are not capable
  of displaying them.  Our eyes are also quite capable of observing a much wider
  range of intensities than our image formats / monitors are capable of
  displaying.

  Tone-mapping is a process that transforms the intensities in the image so that
  the brightest values aren't nearly so far away from the mean.  That way when
  we transform the values into [0-255] we can actually see the entire image.
  There are many ways to perform this process and it is as much an art as a
  science - there is no single "right" answer.  In this homework we will
  implement one possible technique.

  Background Chrominance-Luminance
  ================================

  The RGB space that we have been using to represent images can be thought of as
  one possible set of axes spanning a three dimensional space of color.  We
  sometimes choose other axes to represent this space because they make certain
  operations more convenient.

  Another possible way of representing a color image is to separate the color
  information (chromaticity) from the brightness information.  There are
  multiple different methods for doing this - a common one during the analog
  television days was known as Chrominance-Luminance or YUV.

  We choose to represent the image in this way so that we can remap only the
  intensity channel and then recombine the new intensity values with the color
  information to form the final image.

  Old TV signals used to be transmitted in this way so that black & white
  televisions could display the luminance channel while color televisions would
  display all three of the channels.


  Tone-mapping
  ============

  In this assignment we are going to transform the luminance channel (actually
  the log of the luminance, but this is unimportant for the parts of the
  algorithm that you will be implementing) by compressing its range to [0, 1].
  To do this we need the cumulative distribution of the luminance values.

  Example
  -------

  input : [2 4 3 3 1 7 4 5 7 0 9 4 3 2]
  min / max / range: 0 / 9 / 9

  histo with 3 bins: [4 7 3]

  cdf : [4 11 14]


  Your task is to calculate this cumulative distribution by following these
  steps.


In [22]:
%%cuda --name student_func.cu

#include "utils.h"
__global__ void shmem_reduce(float *d_out, const float* const d_in, int n, int ismax) {
    int tid = threadIdx.x;
    int myid = threadIdx.x + blockDim.x * blockIdx.x;
    extern __shared__ float sdata[];
    if(myid < n)
      sdata[tid] = d_in[myid];
    __syncthreads();

    for(int num = blockDim.x / 2; num > 0; num >>= 1) {
        if(tid < num && myid + num < n) {
            if(ismax)sdata[tid] = max(sdata[tid], sdata[tid + num]);
            else sdata[tid] = min(sdata[tid], sdata[tid + num]);
        }
        __syncthreads();
    }
    if(tid == 0) {
        d_out[blockIdx.x] = sdata[0];
    }
}

__global__ void compute_bins(unsigned int *histo, const size_t numBins,
                             const float* const d_in,
                             float logLumMin, float logLumRange) {
    int myid = threadIdx.x + blockDim.x * blockIdx.x;
    unsigned long myBin = min(static_cast<float>(numBins - 1), 
                    (d_in[myid] - logLumMin) / logLumRange * numBins);
    atomicAdd(&(histo[myBin]), 1);
}

__global__ void hill(unsigned int *d_cdf) {
    int tid = threadIdx.x;
    int blocksize = blockDim.x;
    for(int num = 1; num <= blocksize / 2; num <<= 1) {
        if(tid - num >= 0 && tid < blocksize - 1)
          d_cdf[tid] += d_cdf[tid - num];
        __syncthreads();
    }
    if(tid == blocksize - 1)d_cdf[tid] = 0;
    
    for(int num = 1; num <= blocksize / 2; num <<= 1) {
        if(tid % (num * 2) == 0) {//&& < n
          unsigned int temp = d_cdf[tid + num];
          d_cdf[tid + num] = d_cdf[tid];
          d_cdf[tid] = temp;
        }
        __syncthreads();
    }
    
}

void your_histogram_and_prefixsum(const float* const d_logLuminance,
                                  unsigned int* const d_cdf,
                                  float &min_logLum,
                                  float &max_logLum,
                                  const size_t numRows,
                                  const size_t numCols,
                                  const size_t numBins) {
  const int blockdim = 1 << 10;
  int n = numRows * numCols;
  size_t blocks = (n + blockdim - 1) / blockdim;
  float* d_min, *d_max;
  cudaMalloc(&d_min, n * sizeof(float));
  cudaMalloc(&d_max, n * sizeof(float));

  shmem_reduce<<<blocks, blockdim, blockdim * sizeof(float)>>>(d_min, d_logLuminance, n, 0);
  shmem_reduce<<<1, blocks, blockdim * sizeof(float)>>>(d_min, d_min, n, 0);
  shmem_reduce<<<blocks, blockdim, blockdim * sizeof(float)>>>(d_max, d_logLuminance, n, 1);
  shmem_reduce<<<1, blocks, blockdim * sizeof(float)>>>(d_max, d_max, n, 1);

  cudaMemcpy(&min_logLum, d_min, sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(&max_logLum, d_max, sizeof(float), cudaMemcpyDeviceToHost);

  float logLumRange = max_logLum - min_logLum;
  cudaMemset(d_cdf, 0, numBins * sizeof(unsigned int));
  compute_bins<<<blocks, blockdim>>>(d_cdf, numBins,
                                d_logLuminance, min_logLum, logLumRange);
  unsigned int* h_cdf = (unsigned int*)malloc(blockdim*sizeof(float));
  hill<<<1, blockdim>>>(d_cdf);
}

'File written in /content/drive/MyDrive/udacity-cs344-colab/src/HW3/student_func.cu'

In [23]:
# make the cuda project
!make HW3
print("\n====== RESULT OF HW3 =======\n")

!bin/HW3 ../src/HW3/memorial_png_large.gold

[-14%] Building NVCC (Device) object HW3/CMakeFiles/HW3.dir/HW3_generated_student_func.cu.o
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
[  0%] Linking CXX executable ../bin/HW3
[ 71%] Built target HW3

====== RESULT OF HW3 =======

Your code ran in: 1.668480 msecs.
minmax -4.000000 2.404939
PASS


In [24]:
# plot output images
import matplotlib.pyplot as plt
_,ax = plt.subplots(2,2, dpi=150)

ax[0][0].imshow(plt.imread("../src/HW3/memorial_raw_large.png"))
ax[0][0].set_title("original")
ax[0][0].grid(False)

ax[0][1].imshow(plt.imread("HW3_output.png"))
ax[0][1].set_title("output")
ax[0][1].grid(False)

ax[1][0].imshow(plt.imread("HW3_reference.png"))
ax[1][0].set_title("reference")
ax[1][0].grid(False)

ax[1][1].imshow(plt.imread("HW3_differenceImage.png"))
ax[1][1].set_title("difference")
ax[1][1].grid(False)

plt.show()